# Gradient Boosting Machine from Scratch
***
## Table of Contents
1. [Introduction](#1-introduction)
    - [Advantages](#advantages)
    - [Limitations](#limitations)
    - [Steps](#steps)
2. [Loading Data](#2-loading-data)
3. [Loss Function](#3-loss-function)
    - [Regression](#regression)
    - [Binary Classification](#binary-classification)
    - [Multi-class Classification](#multi-class-classification)
4. [Initialising Model](#4-initialising-model)
5. [Finding the Best Split](#5-finding-the-best-split)
6. [Building Trees](#6-building-trees)
7. [Predictions (Trees)](#7-predictions-trees)
8. [Training Model](#8-training-model)
9. [Final Predictions](#9-final-predictions)
10. [Evaluation Metrics](#10-evaluation-metrics)
    - [Mean Squared Error (MSE)](#mean-squared-error-mse)
    - [Root Mean Squared Error (RMSE)](#root-mean-squared-error-rmse)
    - [Mean Absolute Error (MAE)](#mean-absolute-error-mae)
    - [R-Squared ($R^2$)](#r-squared)
11. [Encapsulation](#11-encapsulation)
12. [Comparison with Scikit-Learn](#12-comparison-with-scikit-learn)
13. [References](#13-references)
***

In [1]:
import numpy as np
import pandas as pd
from typing import Tuple, List, Dict, Any
from sklearn.model_selection import train_test_split
from numpy.typing import NDArray

## 1. Introduction
**Gradient Boosting Machine (GBM)** is an ensemble machine learning model that builds a strong predictive model by sequentially combining multiple weak models (typically decision trees) in a stage-wise manner. The core idea is to iteratively add new models that correct the errors made by the existing ensemble, thereby improving overall predictive accuracy.

Suppose we have a dataset ${(x_i, y_i)}^n_{i=1}$ where $x_i$ are the features and $y_i$ are the target values. The goal of gradient boosting is to find a function $F(x)$ that minimises a given differentiable loss function $L(y, F(x))$:

\begin{align*}
    F(x) = F_0(x) + \sum^{M}_{m=1}\gamma_m h_m(x)
\end{align*}

where:
- $F_0(x)$: Initial model (e.g., the mean of $y$).
- $\gamma_i$: Weight (step size) for the $m$-th weak learner, typically determined by minimising the loss function along the direction of $h_m(x)$.
- $M$: Number of boosting iterations (e.g., the number of weak learners).
- $h_m$: prediction from the $m$-th weak learner (e.g., a decision tree).


GBM can be applied to both regression and classification tasks:
- For **regression**, gradient boosting typically minimises a loss function such as Mean Squared Error(MSE), producing continuous output values.
- For **classification**, the method is adapted to minimise a classification-specific loss function (e.g., logistic loss), yielding class probabilities or class labels.

### Advantages
- High predictive accuracy, even with non-linear relationships.
- Flexible for both regression and classification tasks.
- Can handle numerical and categorical values, as well as missing data without extensive preprocessing.

### Limitations
- Highly prone to overfitting.
- Computationally expensive and time-consuming for large datasets or a large number of trees.
- Sensitive to hypermarameters (e.g., learning rate, number of trees, tree depth, regularisation parameters).
- Poor at handling extrapolation, struggles to predict outcomes outside the range of the training data.

### Steps
1. Initialise the model:
    - Start with a simple model, typically a constant value:
        - For regression: the mean of the target variable.
        - For binary classification: the log-odds of the positive classes.
1. Calculate residuals (Negative Gradients):
    - For each iteration, compute the residuals, which are the negative gradients of the loss function with respect to the current predictions.
    - This step can be generalised to any differentiable loss function, not just the mean squared error.
1. Fit a new weak model to predict the residuals:
    - Train a weak learner (typically a shallow decision tree) to predict the rediduals.
    - The weak learner focuses on correcting the errors made by the current ensemble.
1. Update the model:
    - Add the predictions from the new weak learners to the current model's predictions, scaled by a learning rate (shrinkage parameter).
    - This update moves the ensemble closer to the true values by correcting previous errors.
1. Repeat steps 2-4 for a pre-defined number of iterations.
1. Final prediction:
    - The sum of the initial prediction and the scaled outputs of all weak learners.

## 2. Loading Data
Retrieved from [GitHub - YBI Foundation](https://github.com/YBI-Foundation/Dataset/blob/main/Admission%20Chance.csv)

In [2]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Admission%20Chance.csv')
df.head()

,Serial No,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
feature_names = df.columns[:-1].tolist()  # All columns except the last one

# Check the shape of the data
print(f'Features shape: {X.shape}')
print(f'Target shape: {y.shape}')
print(f'Features: \n{feature_names}')

Features shape: (400, 8)
Target shape: (400,)
Features: 
['Serial No', 'GRE Score', 'TOEFL Score', 'University Rating', ' SOP', 'LOR ', 'CGPA', 'Research']


## 3. Loss Function
For regression, gradient boosting minimises the Mean Squared Error (MSE) by iteratively fitting new models to the negative gradient of the loss function with respect to the current predictions. Although there is no explicit call to MSE, it is integrated inside the gradient calculation.

### Regression
The most common loss function for regression is the mean squared error (MSE):

\begin{align*}
    L(y, \hat y) = \dfrac{1}{n}\sum^{n}_{i=1}(y_i - \hat y_i)^2
\end{align*}

where:
- $y_i$: True value.
- $\hat y_i$: Predicted value.
- $n$: Number of samples.

### Binary Classification
The standard loss function for binary classification in XGBoost is the binary cross-entropy (log loss):

\begin{align*}
    L(y, \hat p) = - \dfrac{1}{n} \sum^{n}_{i=1}\left[y_i \log{(\hat p_i) + (1-y_i) \log{(1- \hat p_i)}} \right]
\end{align*}

where:
- $y_i$: True label ($0$ or $1$).
- $\hat p_i$: Predicted probability for class 1 (after applying the sigmoid function to the raw score).
- $n$: Number of samples.

### Multi-class Classification
For multi-class classification (with $K$ classes), XGBoost uses the multi-class cross-entropy (also called softmax loss):

\begin{align*}
    L(y, \hat P) = - \dfrac{1}{n} \sum^{n}_{i=1} \sum^{K}_{k=1} \mathbb{I}(y_i = k) \log{(\hat p_{ik})}
\end{align*}

where:
- $y_i$: True class label for sample $i$.
- $\hat p_i$: Predicted probability that sample $i$ belongs to class $k$ (output of the softmax function).
- $\mathbb{I}(y_i = k)$: Indicator function, equal to $1$ if $y_i = k$ and $0$ otherwise.
- $n$: Number of samples.

## 4. Initialising Model
First, we need to initialise the model with a constant function that minimises the loss (initial predictions).

\begin{align*}
    F_0(x) = \arg \min_{\gamma}\sum^{n}_{i=1}L(y_i, \gamma)
\end{align*}

For squared error, the best constant is the mean of the target values. Thus,

\begin{align*}
    F_0(x) = \bar y = \dfrac{1}{n}\sum^{n}_{i=1}y_i
\end{align*}

In [4]:
def initialise_model(y: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Initialise predictions with the mean of the target values.

    Parameters:
        y: Target values, shape (n_samples,).

    Returns:
        Array of initial predictions, each set to mean of y.
    """
    return np.full_like(y, np.mean(y), dtype=float)

## 5. Finding the Best Split
The following functions search for the best feature and threshold to split data into two, such that the variance of the resulting child nodes is minimised.

In [5]:
def variance(y: NDArray[np.float64]) -> float:
    """
    Calculate the variance of the target values.

    Args:
        y: Target values.

    Returns:
        float: Variance of y.
    """
    return np.var(y)


def split_dataset(X: NDArray[np.float64], y: NDArray[np.float64], feature_index: int,
                  threshold: float) -> Tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
    """
    Splits the dataset based on a feature and threshold.

    Args:
        X: Feature matrix, shape (n_samples, n_features).
        y: Target values, shape (n_samples,).
        feature_index: Index of the feature to split on.
        threshold: Threshold value for the split.

    Returns:
        Tuple containing X_left, y_left, X_right, y_right after the split.
    """
    left_mask = X[:, feature_index] < threshold
    right_mask = ~left_mask
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]


def best_split(X: NDArray[np.float64], y: NDArray[np.float64], min_samples_leaf: int) -> Tuple[int | None, float | None]:
    """
    Find the best feature and threshold to split the dataset, minimising weighted variance.

    Args:
        X: Feature matrix, shape (n_samples, n_features).
        y: Target values.
        min_samples_leaf: Minimum number of samples required at a leaf node.

    Returns:
        Tuple of (best_feature, best_threshold). Returns (None, None) if no valid split is found.
    """
    m, n = X.shape
    best_feature, best_threshold, best_var = None, None, float('inf')
    for feature in range(n):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            _, y_left, _, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) < min_samples_leaf or len(y_right) < min_samples_leaf:
                continue
            var_left = variance(y_left)
            var_right = variance(y_right)
            var_split = (len(y_left) * var_left + len(y_right) * var_right) / m
            if var_split < best_var:
                best_feature = feature
                best_threshold = threshold
                best_var = var_split
    return best_feature, best_threshold

In [6]:
best_f, best_th = best_split(X, y, 3)
print(f'Best feature(index): {best_f} with its best threshold: {best_th}')

Best feature(index): 6 with its best threshold: 8.74


## 6. Building Trees
This function recursively constructs a regression tree by splitting the data at each node using the best split found, until stopping criteria are met. 

In [7]:
def build_tree(X: NDArray[np.float64], y: NDArray[np.float64], max_depth: int, min_samples_leaf: int, depth: int = 0) -> Dict[str, Any] | float:
    """
    Recursively build a regression tree.

    Args:
        X: Feature matrix.
        y: Target values.
        max_depth: Maximum depth of the tree.
        min_samples_leaf: Minimum samples required at a leaf node.
        depth: Current depth of the tree (default is 0).

    Returns:
        Tree as a nested dictionary, or a float if a leaf node.
    """
    if depth >= max_depth or len(y) <= min_samples_leaf:
        return np.mean(y)
    feature, threshold = best_split(X, y, min_samples_leaf)
    if feature is None:
        return np.mean(y)
    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
    return {
        'feature': feature,
        'threshold': threshold,
        'left': build_tree(X_left, y_left, max_depth, min_samples_leaf, depth + 1),
        'right': build_tree(X_right, y_right, max_depth, min_samples_leaf, depth + 1)
    }

## 7. Predictions (Trees)
The two functions `predict_tree()` and `predict_tree_batch()` are used to make predictions with a regression decision tree represented as a nested dictionary.

- `predict_tree()`: Predicts the output for a single data sample $x$ by traversing the decision tree.
- `predict_tree_batch()`: Generates predictions for a batch of samples by applying the `predict_tree` function to each sample in the input array.

In [8]:
def predict_tree(tree: Dict[str, Any] | float, x: NDArray[np.float64]) -> float:
    """
    Predict the target value for a single sample using the regression tree.

    Args:
        tree: The regression tree or a leaf value.
        x: Feature vector for a single sample.

    Returns:
        float: Predicted value.
    """
    while isinstance(tree, dict):
        if x[tree['feature']] < tree['threshold']:
            tree = tree['left']
        else:
            tree = tree['right']
    return tree


def predict_tree_batch(tree: Dict[str, Any] | float, X: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Predict target values for a batch of samples using the regression tree.

    Args:
        tree: The regression tree or a leaf value.
        X: Feature matrix, shape (n_samples, n_features).

    Returns:
        Predicted values for all samples.
    """
    return np.array([predict_tree(tree, x) for x in X])

## 8. Training Model
During the training process of gradient boosting:
1. Initialise model predictions as $F_0(x) = \bar y$.
2. Boosting loop from $m=1$ to $M$ (number of boosting rounds `n_estimators`):
    - Compute residuals (negative gradients of the MSE loss function).<br><br>
    \begin{align*}
    r_{im} &= - \left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F(x) = F_{m-1(x)}} \\
     &= - \frac{\partial}{\partial \hat y_i} \dfrac{1}{2}(y_i - \hat y_i)^2 = +(y_i - \hat y_i)
    \end{align*}
        Therefore, $r_{im} = y_i - \hat y_i$ where $\hat y$ is the current prediction for sample.

    - Fit tree to residuals.
    - Append the fitted tree.
    - Update predictions.<br><br>
    For each boosting iteration, after fitting the weak learner $h_m(x)$ to the pseudo-residuals, the optimal $\gamma_m$ can be found by solving:
    \begin{align*}
        \gamma_m = \arg \min_{\gamma}\sum^{n}_{i=1}L(y_i, F_{m-1}(x_i) + \gamma h_m(x_i))
    \end{align*}
    However, in practice, $\gamma_m$ is often absorbed into the learning rate (denoted as $\eta$) or set to 1 for simplicity. Thus the update is written as:
    \begin{align*}
        \hat y_i \leftarrow \hat y_i + \eta \cdot h_m(x_i)
    \end{align*}

In [9]:
def gradient_boosting_fit(X: NDArray[np.float64], y: NDArray[np.float64],
                          n_estimators: int = 10, learning_rate: float = 0.1,
                          max_depth: int = 3, min_samples_leaf: int = 1) -> Tuple[float, List[Dict[str, Any] | float], float]:
    """
    Fit a gradient boosting regressor.

    Args:
        X: Feature matrix, shape (n_samples, n_features).
        y: Target values.
        n_estimators: Number of boosting rounds (default is 10).
        learning_rate: Learning rate (default is 0.1).
        max_depth: Maximum depth of each tree (default is 3).
        min_samples_leaf: Minimum samples per leaf (default is 1).

    Returns:
        Tuple containing:
            - Initial prediction (mean of y).
            - List of fitted trees.
            - Learning rate.
    """
    models = []
    predictions = initialise_model(y)
    for m in range(n_estimators):
        residuals = y - predictions
        tree = build_tree(X, residuals, max_depth, min_samples_leaf)
        models.append(tree)
        update = predict_tree_batch(tree, X)
        predictions += learning_rate * update
    return np.mean(y), models, learning_rate

## 9. Final Predictions
The final predictions after $M$ trees is:
\begin{align*}
    F_M(x) = F_0(x) + \sum^{M}_{m=1} \eta \cdot h_m(x)
\end{align*}

where:
- $F_0(x)$: Initial prediction (mean of $y$).
- $h_m(x)$: Prediction of the $m$-th tree.
- $\eta$: Learning rate.

In [10]:
def gradient_boosting_predict(X: NDArray[np.float64], initial_prediction: float, models,
                              learning_rate: float) -> NDArray[np.float64]:
    """
    Predicts using the fitted gradient boosting regressor.

    Args:
        X: Feature matrix, shape (n_samples, n_features).
        initial_prediction: Initial prediction (mean of y from training).
        models: List of fitted regression trees.
        learning_rate: Learning rate used during training.

    Returns:
        Predicted values for all samples.
    """
    y_pred = np.full(X.shape[0], initial_prediction, dtype=float)
    for tree in models:
        y_pred += learning_rate * predict_tree_batch(tree, X)
    return y_pred

## 10. Evaluation Metrics
### Mean Squared Error (MSE)
Mean Squared Error measures the average squared difference between predicted ($\hat y$) and actual ($y$) values. Large errors are penalised heavily. Smaller MSE indicates better predictions.

\begin{align*}
MSE = \dfrac{1}{n} \sum_{i=1}^{n}(\hat y_{i} = y_{i})^2
\end{align*}

In [11]:
def calculate_MSE(y_true: NDArray[np.float64], y_pred: NDArray[np.float64]) -> float:
    return np.mean((y_true - y_pred) ** 2)

### Root Mean Squared Error (RMSE)
Square root of MSE. It provides error in the same unit as the target variable ($y$) and easier to interpret.

\begin{align*}
RMSE = \sqrt{(MSE)}
\end{align*}

In [12]:
def calculate_RMSE(y_true: NDArray[np.float64], y_pred: NDArray[np.float64]) -> float:
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

### Mean Absolute Error (MAE)
Mean Absolute Error measures the average absolute difference between predicted ($\hat y$) and actual ($y$) values. It is less sensitive to outliers than MSE. Smaller MAE indicates better predictions.

\begin{align*}
MAE = \dfrac{1}{n} \sum_{i=1}^{n}|\hat y_{i} = y_{i}|
\end{align*}

In [13]:
def calculate_MAE(y_true: NDArray[np.float64], y_pred: NDArray[np.float64]) -> float:
    return np.mean(np.abs(y_true - y_pred))

<a id="r-squared"></a>
### R-Squared($R^2$)

R-squared indicated the proportion of variance in the dependent variable that is predictable from the independent variables. Value ranges from 0 to 1. Closer to 1 indicates a better fit.



Residual Sum of Squares ($SS_{residual}$): 
\begin{align*}
SS_{residual} = \sum_{i=1}^{n} (y_{i} - \hat y_{i})^{2}
\end{align*}

Total Sum of Squares ($SS_{total}$): 
\begin{align*}
SS_{total} = \sum_{i=1}^{n} (y_{i} - \bar y_{i})^{2}
\end{align*}

$R^2$ is computed as:

\begin{align*}

R^2 = 1 - \dfrac{SS_{residual}}{SS_{total}} = 1 - \dfrac{\sum_{i=1}^{n} (y_{i} - \hat y_{i})^{2}}{\sum_{i=1}^{n} (y_{i} - \bar y_{i})^{2}}

\end{align*}

where:

$y$: Actual target values.

$\bar y$: Mean of the actual target values.

$\hat y$: Precicted target values.

In [14]:
def calculate_r2(y_true: NDArray[np.float64], y_pred: NDArray[np.float64]) -> float:
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)
    ss_residual = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    return r2

In [15]:
def evaluate(y_true: NDArray[np.float64], y_pred: NDArray[np.float64]) -> Tuple[float, float, float, float]:
    """
    Calculate and return evaluation metrics for a regression model, including MSE, RMSE, MAE, and R-squared.

     Args:
        y_true: True labels.
        y_pred: Predicted labels.
        class_names: List of class names. Defaults to None.
    Returns:
        - mse: Mean Squared Error (MSE), indicating the average of the squared differences between predicted and true values.
        - rmse: Root Mean Squared Error (RMSE), indicating the standard deviation of the residuals.
        - mae: Mean Absolute Error (MAE), representing the average absolute difference between predicted and true values.
        - r2: R-squared (coefficient of determination), showing the proportion of variance in the dependent variable that is predictable from the independent variable(s).
    """
    mse = calculate_MSE(y_true, y_pred)
    rmse = calculate_RMSE(y_true, y_pred)
    mae = calculate_MAE(y_true, y_pred)
    r2 = calculate_r2(y_true, y_pred)
    return mse, rmse, mae, r2

## 11. Encapsulation

In [ ]:
class CustomGBRegressor:
    """
    Custom Gradient Boosting for regression with decision trees.
    """

    def __init__(
        self,
        n_estimators: int = 3,
        learning_rate: float = 0.1,
        max_depth: int = 3,
        min_samples_leaf: int = 1
    ):
        """
        Initialises the CustomGBRegressor.

        Args:
            n_estimators: Number of boosting rounds.
            learning_rate: Learning rate (shrinkage).
            max_depth: Maximum depth of each tree.
            min_samples_leaf: Minimum samples per leaf.
        """
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.models: List[Dict[str, Any] | float] = []
        self.initial_prediction: float = 0.0

    def fit(self, X: NDArray[np.float64], y: NDArray[np.float64]) -> None:
        """
        Fit the gradient boosting regressor to the data.

        Args:
            X: Feature matrix, shape (n_samples, n_features).
            y: Target values, shape (n_samples,).
        """
        self.models = []
        self.initial_prediction = float(np.mean(y))
        predictions = np.full_like(y, self.initial_prediction, dtype=float)
        for _ in range(self.n_estimators):
            residuals = y - predictions
            tree = self._build_tree(
                X, residuals, self.max_depth, self.min_samples_leaf)
            self.models.append(tree)
            update = self._predict_tree_batch(tree, X)
            predictions += self.learning_rate * update

    def predict(self, X: NDArray[np.float64]) -> NDArray[np.float64]:
        """
        Predict target values for given feature matrix.

        Args:
            X: Feature matrix, shape (n_samples, n_features).

        Returns:
            Predicted values.
        """
        y_pred = np.full(X.shape[0], self.initial_prediction, dtype=np.float64)
        for tree in self.models:
            y_pred += self.learning_rate * self._predict_tree_batch(tree, X)
        return y_pred

    def _variance(self, y: NDArray[np.float64]) -> float:
        """
        Calculate the variance of the target values.

        Args:
            y: Target values.

        Returns:
            float: Variance of y.
        """
        return np.var(y)

    def _split_dataset(self, X: NDArray[np.float64],
                       y: NDArray[np.float64], feature_index: int,
                       threshold: float) -> Tuple[NDArray[np.float64], NDArray[np.float64], NDArray[np.float64], NDArray[np.float64]]:
        """
        Splits the dataset based on a feature and threshold.

        Args:
            X: Feature matrix, shape (n_samples, n_features).
            y: Target values, shape (n_samples,).
            feature_index: Index of the feature to split on.
            threshold: Threshold value for the split.

        Returns:
            Tuple containing X_left, y_left, X_right, y_right after the split.
        """
        left_mask = X[:, feature_index] < threshold
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def _best_split(self, X: NDArray[np.float64],
                    y: NDArray[np.float64], min_samples_leaf: int) -> Tuple[int | None, float | None]:
        """
        Find the best feature and threshold to split the dataset, minimising weighted variance.

        Args:
            X: Feature matrix, shape (n_samples, n_features).
            y: Target values.
            min_samples_leaf: Minimum number of samples required at a leaf node.

        Returns:
            Tuple of (best_feature, best_threshold). Returns (None, None) if no valid split is found.
        """
        m, n = X.shape
        best_feature, best_threshold, best_var = None, None, float('inf')
        for feature in range(n):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                _, y_left, _, y_right = self._split_dataset(
                    X, y, feature, threshold)
                if len(y_left) < min_samples_leaf or len(y_right) < min_samples_leaf:
                    continue
                var_left = self._variance(y_left)
                var_right = self._variance(y_right)
                var_split = (len(y_left) * var_left +
                             len(y_right) * var_right) / m
                if var_split < best_var:
                    best_feature = feature
                    best_threshold = threshold
                    best_var = var_split
        return best_feature, best_threshold

    def _build_tree(self, X: NDArray[np.float64], y: NDArray[np.float64],
                    max_depth: int, min_samples_leaf: int, depth: int = 0) -> Dict[str, Any] | float:
        """
        Recursively build a regression tree.

        Args:
            X: Feature matrix.
            y: Target values.
            max_depth: Maximum depth of the tree.
            min_samples_leaf: Minimum samples required at a leaf node.
            depth: Current depth of the tree (default is 0).

        Returns:
            Tree as a nested dictionary, or a float if a leaf node.
        """
        if depth >= max_depth or len(y) <= min_samples_leaf:
            return float(np.mean(y))
        feature, threshold = self._best_split(X, y, min_samples_leaf)
        if feature is None:
            return float(np.mean(y))
        X_left, y_left, X_right, y_right = self._split_dataset(
            X, y, feature, threshold)
        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X_left, y_left, max_depth, min_samples_leaf, depth + 1),
            'right': self._build_tree(X_right, y_right, max_depth, min_samples_leaf, depth + 1)
        }

    def _predict_tree(self, tree: Dict[str, Any] | float, x: NDArray[np.float64]) -> float:
        """
        Predict the target value for a single sample using the regression tree.

        Args:
            tree: The regression tree or a leaf value.
            x: Feature vector for a single sample.

        Returns:
            float: Predicted value.
        """
        while isinstance(tree, dict):
            if x[tree['feature']] < tree['threshold']:
                tree = tree['left']
            else:
                tree = tree['right']
        return float(tree)

    def _predict_tree_batch(self, tree: Dict[str, Any] | float, X: NDArray[np.float64]) -> NDArray[np.float64]:
        """
        Predict target values for a batch of samples using the regression tree.

        Args:
            tree: The regression tree or a leaf value.
            X: Feature matrix, shape (n_samples, n_features).

        Returns:
            Predicted values for all samples.
        """
        return np.array([self._predict_tree(tree, x) for x in X], dtype=np.float64)

In [17]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Instantiate and fit the model
model_custom = CustomGBRegressor(n_estimators=200, learning_rate=0.1,
                                 max_depth=2, min_samples_leaf=1)
model_custom.fit(X_train, y_train)

# Predict and evaluate
y_pred_custom = model_custom.predict(X_test)
mse_custom, rmse_custom, mae_custom, r2_custom = evaluate(
    y_test, y_pred_custom)
print(f'MSE (Custom): {mse_custom:.4f}')
print(f'RMSE (Custom): {rmse_custom:.4f}')
print(f'MAE (Custom): {mae_custom:.4f}')
print(f'R-Squared (Custom): {r2_custom:.4f}')
print('----------')

MSE (Custom): 0.0038
RMSE (Custom): 0.0614
MAE (Custom): 0.0427
R-Squared (Custom): 0.8541
----------


## 12. Comparison with Scikit-Learn

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score

# Instantiate and fit the sklearn model
sklearn_gbm = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=2,
    min_samples_leaf=1,
    random_state=42
)
sklearn_gbm.fit(X_train, y_train)

# Predict and evaluate
y_pred_sklearn = sklearn_gbm.predict(X_test)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
rmse_sklearn = root_mean_squared_error(y_test, y_pred_sklearn)
mae_sklearn = mean_absolute_error(y_test, y_pred_sklearn)
r2_sklearn = r2_score(y_test, y_pred_sklearn)
print(f'MSE (SK): {mse_sklearn:.4f}')
print(f'RMSE (SK): {rmse_sklearn:.4f}')
print(f'MAE (SK): {mae_sklearn:.4f}')
print(f'R-Squared (SK): {r2_sklearn:.4f}')
print('----------')

MSE (SK): 0.0039
RMSE (SK): 0.0625
MAE (SK): 0.0449
R-Squared (SK): 0.8489
----------


## 13. References
1. Andreas Mueller. (2020). *Applied ML 2020 - 08 - Gradient Boosting.* <br>
https://www.youtube.com/watch?v=yrTW5YTmFjw

1. Bex Tuychiev. (2023). *A Guide to The Gradient Boosting Algorithm.* <br>
https://www.datacamp.com/tutorial/guide-to-the-gradient-boosting-algorithm

1. DataMListic. (2023). *Gradient Boosting with Regression Trees Explained* [YouTube Video]. <br>
https://youtu.be/lOwsMpdjxog

1. scikit-learn. (n.d.). *GradientBoostingRegressor — scikit-learn API Reference.* <br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

1. scikit-learn. (n.d.). *Gradient Boosting Regression — scikit-learn Examples.* <br>
https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

1. StatQuest with Josh Starmer. (2019). *Gradient Boost Part 1 (of 4): Regression Main Ideas* [YouTube Video]. <br>
https://youtu.be/3CC4N4z3GJc

1. StatQuest with Josh Starmer. (2019). *Gradient Boost Part 2 (of 4): Regression Details* [YouTube Video]. <br>
https://youtu.be/2xudPOBz-vs

1. Terence Parr and Jeremy Howard. (n.d.). *How to explain gradient boosting.* <br>
https://explained.ai/gradient-boosting/index.html

1. Tomonori Masui. (2022). *All You Need to Know about Gradient Boosting Algorithm − Part 1. Regression.* <br>
https://medium.com/data-science/all-you-need-to-know-about-gradient-boosting-algorithm-part-1-regression-2520a34a502